# Sampling

Content credits on the [Acknowledgments Page](https://ds100.org/sp23/acks/).

Updated by Joseph Gonzalez, Dominic Liu, Fernando Pérez.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_theme(style='darkgrid', font_scale = 1.5,
              rc={'figure.figsize':(7,5)})

rng = np.random.default_rng()

## Barbie v. Oppenheimer

To study how various sampling strategies work we will use a (fictional)  **census** -- a complete survey of all Berkeley residents (our population). For the purposes of this fictional demo, assume:
* `wears_birkenstocks` indicates if a resident identifies as male.
* There are only two movies they can watch on July 21st: Barbie and Oppenheimer.
* Every resident watches a movie (either Barbie or Oppenheimer) on July 21st.


In [ ]:
census = pd.read_csv("movie_census.csv")
census['Barbie'] = census['movie'] == 'Barbie'
census

What fraction of Berkeley residents chose Barbie?

In [ ]:
actual_barbie = census["Barbie"].mean()
actual_barbie

This is the **actual outcome** of the competition. Based on this result, Barbie would win. How did our sample of retirees do?

## Convenience sample: _Undergrads in Prof. Gonzalez OH_

In [ ]:
undergrads = census[(18 <= census['age']) & (census['age'] <= 22)].sample(10, replace=False)
undergrads["Barbie"].mean()

Based on this result, we would have predicted that Oppenheimer would win! What happened?

1. Is the sample too small / noisy?

In [ ]:
len(undergrads)

In [ ]:
print("Percent of Berkeley:", len(undergrads)/len(census) * 100)

## Convenience sample: _Elderly at a Campus Event_

In [ ]:
elderly = census[census['age'] >= 65].sample(100)
elderly["Barbie"].mean()

Based on this result, we would have predicted that Oppenheimer would win! What happened?

1. Is the sample too small / noisy?

In [ ]:
len(elderly)

In [ ]:
print("Percent of Berkeley:", len(elderly)/len(census) * 100)

### Check for bias

Let us aggregate all choices by age and visualize the fraction of Barbie views, split by gender.

In [ ]:
votes_by_barbie = census.groupby(["age","wears_birkenstocks"]).agg("mean", numeric_only=True).reset_index()
votes_by_barbie

In [ ]:
import plotly.express as px
px.scatter(votes_by_barbie, x = "age", y = "Barbie", 
           color = "wears_birkenstocks",
           title= "Preferences by Demographics")

* We see that retirees (in Berkeley) tend to watch Oppenheimer.
* We also see that residents who don't routinely wear Birkenstocks tend to prefer Barbie (nothing wrong with Birkenstocks).

## Simple Random Sample

What if we instead took a simple random sample (SRS) to collect our sample?

Suppose we took an SRS of the same size as our undergrad sample:

In [ ]:
## By default, replace = False
n = 800
random_sample = census.sample(n, replace = False)

random_sample["Barbie"].mean()

This is very close to the actual vote!

In [ ]:
actual_barbie

It turns out that we can get similar results with a **much smaller sample size**, say, 800:

In [ ]:
n = 800
random_sample = census.sample(n, replace = False)

# Compute the sample average and the resulting relative error
sample_barbie = random_sample["Barbie"].mean()
err = abs(sample_barbie-actual_barbie)/actual_barbie

# We can print output with Markdown formatting too...
from IPython.display import Markdown
Markdown(f"**Actual** = {actual_barbie:.4f}, **Sample** = {sample_barbie:.4f}, "
         f"**Err** = {100*err:.2f}%.")

We'll learn how to choose this number when we (re)learn the Central Limit Theorem later in the semester.

### Quantifying chance error

In our SRS of size 800, what would be our chance error?

Let's simulate 1000 versions of taking the 800-sized SRS from before:

In [ ]:
nrep = 1000   # number of simulations
n = 800       # size of our sample
poll_result = []
for i in range(0, nrep):
    random_sample = census.sample(n, replace = False)
    poll_result.append(random_sample["Barbie"].mean())

Visualizing the distribution of outcomes:

In [ ]:
fig = px.histogram(poll_result, histnorm='probability density', nbins=50)
fig.add_vline(x=actual_barbie, line_width=3, line_dash="dash", line_color="orange")
fig.update_layout(showlegend=False)

# Add Kernel Density Estimate curve
from scipy import stats
from plotly import graph_objects as go
x = np.linspace(min(poll_result), max(poll_result), 100)
fig.add_trace(go.Scatter(
    x=x, 
    y=stats.gaussian_kde(poll_result)(x), # Library for KDE (auto selects bandwidth)
    mode='lines', line=dict(color='red', width=3)) # Formatting
    )


Using seaborn instead:

In [ ]:
sns.histplot(poll_result, stat='density', kde=True);
plt.axvline(actual_barbie, color='orange', linestyle='dashed', linewidth=2)

What fraction of these simulated samples would have predicted Barbie?

In [ ]:
poll_result = pd.Series(poll_result)
np.sum(poll_result > 0.5)/1000

<br><br>

**Return to Slides**

<br><br>

---

## Simulating from a Multinomial Distribution

Sometimes instead of having individual reports in the population, we have **aggregate** statistics. For example, we could have only learned that 53\% of election voters voted Democrat. Even so, we can still simulate probability samples if we assume the population is large.

Specifically, we can use **multinomial** probabilities to simulate random samples **with replacement**.

### Marbles

Suppose we have a very large bag of marbles with the following statistics:
* 60\% blue
* 30\% green
* 10\% red

We then draw 100 marbles from this bag at random with replacement.

In [ ]:
np.random.multinomial(100, [0.60, 0.30, 0.10])

We can repeat this simulation multiple times, say 20:

In [ ]:
np.random.multinomial(100, [0.60, 0.30, 0.10], size=20)